In [1]:
import pandas as pd
import numpy as np
import random
import math
import time

In [2]:
wzór = pd.DataFrame(columns=["Strategia", "Pamięć", "Tryb życia", "Punkty", "Wiek", "Status" "Fenotyp A", "Fenotyp B", "Lokalizacja", "Zdrajcy", "Powód śmierci", "Potomkowie"])
wzór

,Strategia,Pamięć,Tryb życia,Punkty,Wiek,StatusFenotyp A,Fenotyp B,Lokalizacja,Zdrajcy,Powód śmierci,Potomkowie


In [3]:
#Ustalenie liczby graczy
n = 20
#Ustalenie liczby iteracji
t = 10
#Ustalenie limitu długości życia
xt = 20

In [4]:
#tworzenie indeksów
Gra = np.zeros((n, 1))

index = np.arange(n)
Gra[:,0] = index

#### Możliwe strategie bazowe:
- egoistyczna (0, E) - zawsze zdrada
- kooperatywna (1, K) - współpraca z każdym
- podobnych sobie (2, P) - współpraca z graczami podzielającymi w większości ten sam fenotyp, zdrada innych
- różnych sobie (3, R) - współpraca z graczami o innym fenotypie, zdrada graczy o podobnym fenotypie
- familijna (4, F) - współpraca tylko z własnymi rodzicami i potomkami
- asertywna (5, A) - otwarcie na współpracę z każdym, dopóki liczba punktów nie spadnie poniżej progu krytycznego

In [5]:
l_s = 6
strategia = np.random.randint(l_s, size=(n, 1))
Gra = np.hstack((Gra, strategia))

In [6]:
print(f"Strategia egoistyczna występuje {np.count_nonzero(Gra[:, 1] == 0)} razy.")
print(f"Strategia kooperacyjna występuje {np.count_nonzero(Gra[:, 1] == 1)} razy.")
print(f"Strategia podobnych sobie występuje {np.count_nonzero(Gra[:, 1] == 2)} razy.")
print(f"Strategia różnych sobie występuje {np.count_nonzero(Gra[:, 1] == 3)} razy.")
print(f"Strategia familijna występuje {np.count_nonzero(Gra[:, 1] == 4)} razy.")
print(f"Strategia asertywna występuje {np.count_nonzero(Gra[:, 1] == 5)} razy.")

Strategia egoistyczna występuje 3 razy.
Strategia kooperacyjna występuje 5 razy.
Strategia podobnych sobie występuje 4 razy.
Strategia różnych sobie występuje 4 razy.
Strategia familijna występuje 3 razy.
Strategia asertywna występuje 1 razy.


#### Możliwa pamięć:
- brak (0, B) - gracz prostępuje zawsze tak samo, niezależnie od poprzednich działań innych graczy
- długa (1, D) - gracz jeżeli zostanie zdradzony, nigdy już nie podejmie współpracy ze zdrajcą

In [7]:
l_p = 2
pamięć = np.random.randint(l_p, size=(n, 1))
Gra = np.hstack((Gra, pamięć))

In [8]:
print(f"Braki pamięci posiada: {np.count_nonzero(Gra[:, 2] == 0)} graczy.")
print(f"Pamięć długą posiada: {np.count_nonzero(Gra[:, 2] == 1)} graczy.")

Braki pamięci posiada: 7 graczy.
Pamięć długą posiada: 13 graczy.


#### Możliwy tryb życia:
- koczowniczy [0] - jednostka zmienia swoje położenie co turę
- zakorzeniony [1] - jednostka nigdy nie zmienia swojego położenia w trakcie gry
- osiadły [2] - jednostka nie zmienia swojego położenia w trakcie gry, chyba że na danym terenie występuje przeludnienie (dla limitu 1000, pop = 700) (w planach)

In [9]:
l_t = 2
tryb_życia= np.random.randint(l_t, size=(n, 1))
Gra = np.hstack((Gra, tryb_życia))

In [10]:
print(f"Koczowniczy tryb życia prowadzi: {np.count_nonzero(Gra[:, 3] == 0)} graczy.")
print(f"Zakorzeniony tryb życia prowadzi: {np.count_nonzero(Gra[:, 3] == 1)} graczy.")

Koczowniczy tryb życia prowadzi: 9 graczy.
Zakorzeniony tryb życia prowadzi: 11 graczy.


#### Status początkowy
Wszystkie jednostki rozpoczynają ze 100 punktami, wiekiem 0 oraz statusem żywym (0). Zmniejszenie się liczby punktów poniżej 0 oznacza śmierć (1).

In [11]:
pkt = np.full_like(Gra[:, :1], 100)
Gra = np.hstack((Gra, pkt))

In [12]:
wiek = np.full_like(Gra[:, :1], 0)
Gra = np.hstack((Gra, wiek))

In [13]:
status = np.full_like(Gra[:, :1], 0)
Gra = np.hstack((Gra, status))

In [14]:
Fenotyp = np.random.randint(2, size=(n, 1))
Gra = np.hstack((Gra, Fenotyp))

FenotypA = np.full_like(Gra[:, :1], 0)
Gra = np.hstack((Gra, FenotypA))
FenotypA = np.where(Gra[:,7] == 0, 100, Gra[:, 8])
Gra[:,8] = FenotypA

FenotypB = np.full_like(Gra[:, :1], 0)
Gra = np.hstack((Gra, FenotypB))
FenotypB = np.where(Gra[:,7] == 1, 100, Gra[:, 9])
Gra[:,9] = FenotypB

Gra = np.delete(Gra, 7, axis=1)

In [15]:
print(f"Fenotyp A występuje {np.count_nonzero(Gra[:, 7] == 100)} razy.\nFenotyp B występuje {np.count_nonzero(Gra[:, 8] == 100)} razy.")

Fenotyp A występuje 10 razy.
Fenotyp B występuje 10 razy.


#### Lokalizacja
Jednoski zostaną rozrzucone w 4 przestrzeniach oddzielonych barierami naturalnymi od reszty lokalizacji.

In [16]:
Lokalizacja = np.random.randint(4, size=(n, 1))
Gra = np.hstack((Gra, Lokalizacja))

In [17]:
print(f"Liczba graczy w danej lokalicjacji: 0: {np.count_nonzero(Gra[:, 9] == 0)}")
print(f"Liczba graczy w danej lokalicjacji: 1: {np.count_nonzero(Gra[:, 9] == 1)}")
print(f"Liczba graczy w danej lokalicjacji: 2: {np.count_nonzero(Gra[:, 9] == 2)}")
print(f"Liczba graczy w danej lokalicjacji: 3: {np.count_nonzero(Gra[:, 9] == 3)}")

Liczba graczy w danej lokalicjacji: 0: 8
Liczba graczy w danej lokalicjacji: 1: 5
Liczba graczy w danej lokalicjacji: 2: 5
Liczba graczy w danej lokalicjacji: 3: 2


#### Powód śmierci
Możliwe wartości: 0 - status żywy, 1 - naturalny, 2 - walka

In [18]:
Powód = np.full_like(Gra[:, :1], 0)
Gra = np.hstack((Gra, Powód))

In [19]:
Gra

array([[  0.,   0.,   1.,   0., 100.,   0.,   0.,   0., 100.,   2.,   0.],
       [  1.,   0.,   0.,   0., 100.,   0.,   0., 100.,   0.,   2.,   0.],
       [  2.,   2.,   0.,   1., 100.,   0.,   0., 100.,   0.,   0.,   0.],
       [  3.,   5.,   1.,   0., 100.,   0.,   0., 100.,   0.,   1.,   0.],
       [  4.,   3.,   0.,   0., 100.,   0.,   0., 100.,   0.,   1.,   0.],
       [  5.,   3.,   1.,   0., 100.,   0.,   0., 100.,   0.,   0.,   0.],
       [  6.,   1.,   1.,   1., 100.,   0.,   0.,   0., 100.,   1.,   0.],
       [  7.,   4.,   0.,   0., 100.,   0.,   0.,   0., 100.,   0.,   0.],
       [  8.,   3.,   0.,   1., 100.,   0.,   0., 100.,   0.,   2.,   0.],
       [  9.,   1.,   0.,   1., 100.,   0.,   0.,   0., 100.,   1.,   0.],
       [ 10.,   2.,   1.,   1., 100.,   0.,   0.,   0., 100.,   2.,   0.],
       [ 11.,   4.,   1.,   0., 100.,   0.,   0., 100.,   0.,   2.,   0.],
       [ 12.,   1.,   1.,   1., 100.,   0.,   0.,   0., 100.,   0.,   0.],
       [ 13.,   2.,   1.,

#### Pamięć graczy
W osobnych strukturach są zapisywane informacje zapamiętane przez graczy. Gracze mają możliwość zapamiętania zdrajców, rodziców i potomków.

In [20]:
Gra_potomkowie = np.zeros((n, n))
Gra_rodzice = np.zeros((n, n))
Gra_zdrajcy = np.zeros((n, n))

#### Zasady gry
Bazą do zasad gry jest dylemat więźnia. Punktacja jest zależna od malejącej funkcji populacji. Punktacja scenariuszy:
- Obopólna pomoc: obie strony otrzymują po F pkt. (Suma pkt. 2F)
- Obopólna zdrada: obie strony tracą po F-11 pkt. (Suma pkt. 2F-22)
- Jednostronna zdrada: strona zdradzająca zyskuje F+10 pkt., strona zdradzona zyskuje traci F-20 pkt. (Suma pkt. 2F-10)

Wzór na funkcję punktów:\
F(Pop) = 10,05-0.0043*Pop-0.00000868*Pop^2+0.000004*Pop^3\
Liczba punktów jest zaokrąglana do dwóch miejsc po przecinku.

In [21]:
#Macierz sąsiedztwa przedstawiająca prawdopodobieństwo zmiany lokalizacji dla koczowników
Mapa = np.array([[0, 0.5, 0.5, 0],
     [0.5, 0, 0, 0.5],
     [0.5, 0, 0, 0.5],
     [0, 0.5, 0.5, 0]])
#Poniżej macierz sąsiedztwa dla przestrzeni z 9 lokalizacjami
#Mapa = np.array([[0, 0.5, 0, 0.5, 0, 0, 0, 0, 0],
#                 [1/3, 0, 1/3, 0, 1/3, 0, 0, 0, 0],
#                 [0, 0.5, 0, 0, 0, 0.5, 0, 0, 0],
#                 [1/3, 0, 0, 0, 1/3, 0, 1/3, 0, 0],
#                 [0, 0.25, 0, 0.25, 0, 0.25, 0, 0.25, 0],
#                 [0, 0, 1/3, 0, 1/3, 0, 0, 0, 1/3],
#                 [0, 0, 0, 0.5, 0, 0, 0, 0.5, 0],
#                 [0, 0, 0, 0, 1/3, 0, 1/3, 0, 1/3],
#                 [0, 0, 0, 0, 0, 0.5, 0, 0.5, 0]])
#Pojemność środowiska
Limit=1000
#Tempo wzrostu populacji
r_pop=0.2
#Prawdopodobieństwo mutacji
m = 0.02

In [22]:
#funkcja definiująca postępowanie drugiego gracza w oparciu o strategie
def stg():
    global g2
    #Pamięć B
    if Gra[j, 2] == 0:
        #Stg K
        if Gra[j, 1] == 1:
            g2 = 1
        #Stg E
        elif Gra[j, 1] == 0:
            g2 = 0
        #Stg P
        elif Gra[j, 1] == 2:
            if Gra[j, 7]==Gra[j, 8]:
                g2 = 1
            else:
                N_i2 = np.argmax(Gra[i, 7:9]) + 7
                N_j2 = np.argmax(Gra[j, 7:9]) + 7
                if N_j2 == N_i2:
                    g2 = 1
                else:
                    g2 = 0
        #Stg R
        elif Gra[j, 1] == 3:
            Z_j2 = np.argmax(Gra[i, 7:9]) + 7
            Z_i2 = np.argmax(Gra[j, 7:9]) + 7
            if Z_j2 != Z_i2:
                g2 = 1
            else:
                g2 = 0
        #Str F
        elif Gra[j, 1] == 4:
            if Gra_potomkowie[j, i] == 1 or Gra_rodzice[i, j] == 1:
                g2 = 1
            else:
                g2 = 0
        #Str A
        elif Gra[j, 1] == 5:
            pop_pkt = np.count_nonzero((Gra[:, 9] == Gra[j, 9]) & (Gra[:, 6] == 0))
            f_pkt = round(10.05-0.0043*pop_pkt-0.0000175*pop_pkt**2+0.000004*pop_pkt**3, 2)
            if Gra[j, 4] > (f_pkt-20)*3:
                g2 = 1
            else:
                g2 = 0
    #Pamięć D
    elif Gra[j, 2] == 1:
        if Gra_zdrajcy[j,i] == 1:
            g2=0
        else:
            #Stg K
            if Gra[j, 1] == 1:
                g2 = 1
            #Stg E
            elif Gra[j, 1] == 0:
                g2 = 0
            #Stg P
            elif Gra[j, 1] == 2:
                if Gra[j, 7]==Gra[j, 8]:
                    g2 = 1
                else:
                    N_i2 = np.argmax(Gra[i, 7:9]) + 7
                    N_j2 = np.argmax(Gra[j, 7:9]) + 7
                    if N_j2 == N_i2:
                        g2 = 1
                    else:
                        g2 = 0
            #Stg R
            elif Gra[j, 1] == 3:
                Z_j2 = np.argmax(Gra[i, 7:9]) + 7
                Z_i2 = np.argmax(Gra[j, 7:9]) + 7
                if Z_j2 != Z_i2:
                    g2 = 1
                else:
                    g2 = 0
            #Str F
            elif Gra[j, 1] == 4:
                if Gra_potomkowie[j, i] == 1 or Gra_rodzice[i, j] == 1:
                    g2 = 1
                else:
                    g2 = 0
            #Str A
            elif Gra[j, 1] == 5:
                pop_pkt = np.count_nonzero((Gra[:, 9] == Gra[j, 9]) & (Gra[:, 6] == 0))
                f_pkt = round(10.05-0.0043*pop_pkt-0.0000175*pop_pkt**2+0.000004*pop_pkt**3, 2)
                if Gra[j, 4] > (f_pkt-20)*3:
                    g2 = 1
                else:
                    g2 = 0

In [23]:
Lista_wyniki = []
start_time = time.time()
Wyniki_całość = pd.DataFrame(Gra)
Wyniki_całość["Runda"] = 0
global Wyniki_r, Lista_wyniki, Gra
for _ in range(t):
    for i in range(0, len(Gra)):
        #Sprawdzenie czy gracz i nie jest martwy
        if Gra[i, 6] == 1:
            continue
        for j in range(0, i):
            #Sprawdzenie czy gracz j nie jest martwy
            if Gra[j, 6] == 1:
                continue
            #Upewnienie się czy gracz i w trakcie rozgrywki nie stał się martwy
            if Gra[i, 6] == 1:
                continue
            #Sprawdzanie zgodności lokalizacji
            if Gra[i, 9] != Gra[j, 9]:
                continue
            #Pamięć B
            if Gra[i, 2] == 0:
                #Stg K
                if Gra[i, 1] == 1:
                    g1 = 1
                    stg()
                #Stg E
                elif Gra[i, 1] == 0:
                    g1 = 0
                    stg()
                #Stg P
                elif Gra[i, 1] == 2:
                    if Gra[i, 7]==Gra[i, 8]:
                        g1 = 1
                    else:
                        N_i1 = np.argmax(Gra[i, 7:9]) + 7
                        N_j1 = np.argmax(Gra[j, 7:9]) + 7
                        if N_j1 == N_i1:
                            g1 = 1
                        else:
                            g1 = 0
                    stg()
                #Stg R
                elif Gra[i, 1] == 3:
                    Z_j1 = np.argmax(Gra[i, 7:9]) + 7
                    Z_i1 = np.argmax(Gra[j, 7:9]) + 7
                    if Z_j1 != Z_i1:
                        g1 = 1
                    else:
                        g1 = 0
                    stg()
                #Stg F
                elif Gra[i, 1] == 4:
                    if Gra_potomkowie[i, j] == 1 or Gra_rodzice[j, i] == 1:
                        g1 = 1
                    else:
                        g1 = 0
                    stg()
                #Str A
                elif Gra[i, 1] == 5:
                    pop_pkt = np.count_nonzero((Gra[:, 9] == Gra[i, 9]) & (Gra[:, 6] == 0))
                    f_pkt = round(10.05-0.0043*pop_pkt-0.0000175*pop_pkt**2+0.000004*pop_pkt**3, 2)
                    if Gra[i, 4] > (f_pkt-20)*3:
                        g1 = 1
                    else:
                        g1 = 0
                    stg()
            #Pamięć D
            elif Gra[i, 2] == 1:
                if Gra_zdrajcy[i,j] == 1:
                    g1=0
                    stg()
                else:
                    #Stg K
                    if Gra[i, 1] == 1:
                        g1 = 1
                        stg()
                    #Stg E
                    elif Gra[i, 1] == 0:
                        g1 = 0
                        stg()
                    #Stg P
                    elif Gra[i, 1] == 2:
                        if Gra[i, 7]==Gra[i, 8]:
                            g1 = 1
                        else:
                            N_i1 = np.argmax(Gra[i, 7:9]) + 7
                            N_j1 = np.argmax(Gra[j, 7:9]) + 7
                            if N_j1 == N_i1:
                                g1 = 1
                            else:
                                g1 = 0
                        stg()
                    #Stg R
                    elif Gra[i, 1] == 3:
                        Z_j1 = np.argmax(Gra[i, 7:9]) + 7
                        Z_i1 = np.argmax(Gra[j, 7:9]) + 7
                        if Z_j1 != Z_i1:
                            g1 = 1
                        else:
                            g1 = 0
                        stg()
                    #Stg F
                    elif Gra[i, 1] == 4:
                        if Gra_potomkowie[i, j] == 1 or Gra_rodzice[j, i] == 1:
                            g1 = 1
                        else:
                            g1 = 0
                        stg()
                    #Stg A
                    elif Gra[i, 1] == 5:
                        pop_pkt = np.count_nonzero((Gra[:, 9] == Gra[i, 9]) & (Gra[:, 6] == 0))
                        f_pkt = round(10.05-0.0043*pop_pkt-0.0000175*pop_pkt**2+0.000004*pop_pkt**3, 2)
                        if Gra[i, 4] > (f_pkt-20)*3:
                            g1 = 1
                        else:
                            g1 = 0
                        stg()
            #Wzór na funkcję punktów
            pop_pkt = np.count_nonzero((Gra[:, 9] == Gra[i, 9]) & (Gra[:, 6] == 0))
            f_pkt = round(10.05-0.0043*pop_pkt-0.0000175*pop_pkt**2+0.000004*pop_pkt**3, 2)
            #Przyznanie punktów
            if g1 == 1:
                if g2 == 1:
                    Gra[i, 4] += f_pkt
                    Gra[j, 4] += f_pkt
                else:
                    Gra[i, 4] += f_pkt-20
                    Gra[j, 4] += f_pkt+10
            else:
                if g2 == 1:
                    Gra[i, 4] += f_pkt+10
                    Gra[j, 4] += f_pkt-20
                else:
                    Gra[i, 4] += f_pkt-11
                    Gra[j, 4] += f_pkt-11
            if Gra[i, 4] <= 0:
                Gra[i, 6] = 1
                Gra[i, 10] = 2
            if Gra[j, 4] <= 0:
                Gra[j, 6] = 1
                Gra[j, 10] = 2
            #Przypisanie statusu zdrajcy
            if g2 == 0:
                if Gra_zdrajcy[i,j] == 1:
                    continue
                else:
                    Gra_zdrajcy[i,j] = 1
            if g1 == 0:
                if Gra_zdrajcy[j,i] == 1:
                    continue
                else:
                    Gra_zdrajcy[j,i] = 1
            #Wprowadzenie zachowań koczowniczych gracz j
            if Gra[j, 3] == 0:
                Gra[j, 9] = np.random.choice(np.arange(0, 4), p=Mapa[int(Gra[j,9]), :])
        #Wprowadzenie zachowań koczowniczych gracz i
        if Gra[i, 3] == 0:
            Gra[i, 9] = np.random.choice(np.arange(0, 4), p=Mapa[int(Gra[i, 9]), :])
    print(f"Runda {_+1}")
    #Wprowadzenie wieku
    Gra[Gra[:,6] == 0, 5] += 1
    #Wprowadzenie statusu śmierci
    Gra[Gra[:, 5] == xt, 6] = 1
    Gra[Gra[:, 5] == xt, 10] = 1
    #Wprowadzenie dziedziczenia
    for i in range(len(Gra)):
        if Gra[i,6] == 1 and Gra[i,10] == 1 and Gra[i,4] > 0:
            potomkowie = np.where(Gra_potomkowie[i] == 1)[0]
            liczba_potomków = np.count_nonzero(Gra_potomkowie[i] == 1)
            if liczba_potomków > 0:
                punkty = Gra[i,4]/liczba_potomków
                for potomek in potomkowie:
                    Gra[int(potomek), 4] += punkty
            Gra[i, 4] = 0
    #Wprowadzenie reprodukcji
    for q in range(0, 4):
        N = Gra[Gra[:, 9] == q, :]
        N = len(N[:, 4] > 100)
        #math.ceil zaokrągla w górę
        Nd = math.ceil(N*r_pop*(1-(N/Limit)))
        Pop_r = Nd*2
        Kandydaci = Gra[Gra[:,9] == q, :]
        Kandydaci = Kandydaci[Kandydaci[:,4] > 100,:]
        try:
            #sample() losuje określoną liczbę kandydatów, prawdopodobieństwo wylosowania jest mierzone liczbą posiadanych punktów
            prawdopodobieństwo = Kandydaci[:,4]/np.sum(Kandydaci[:,4])
            indeksy = np.random.choice(Kandydaci[:, 0], size=Pop_r, replace=False, p=prawdopodobieństwo)
            #Pary = [(indeksy[i], indeksy[i+1]) for i in range(0, len(indeksy), 2)]
            for q2 in range(0, len(indeksy), 2):
                nowy_index = int(max(Gra[:,0]))+1
                nowa_strategia = np.random.choice([Gra[int(indeksy[q2]), 1],
                                                   Gra[int(indeksy[q2+1]), 1],
                                                   np.random.randint(l_s)],
                                                  p = [(1-m)/2, (1-m)/2, m])                  
                nowa_pamięć = np.random.choice([Gra[int(indeksy[q2]), 2],
                                                Gra[int(indeksy[q2+1]), 2],
                                                np.random.randint(l_p)],
                                                p = [(1-m)/2, (1-m)/2, m])
                nowy_tż = np.random.choice([Gra[int(indeksy[q2]), 3],
                                            Gra[int(indeksy[q2+1]), 3],
                                            np.random.randint(l_t)],
                                            p = [(1-m)/2, (1-m)/2, m])              
                Gra[int(indeksy[q2]), 4] = Gra[int(indeksy[q2]), 4] - 50                
                Gra[int(indeksy[q2+1]), 4] = Gra[int(indeksy[q2+1]), 4] - 50                
                nowy_fenotyp_a = (Gra[int(indeksy[q2]), 7] + Gra[int(indeksy[q2+1]), 7]) / 2                
                nowy_fenotyp_b = (Gra[int(indeksy[q2]), 8] + Gra[int(indeksy[q2+1]), 8]) / 2                
                nowa_lokalizacja = Gra[int(indeksy[q2]), 9]                
                Gra = np.append(Gra,
                                np.array([[nowy_index,
                                                nowa_strategia,
                                                nowa_pamięć,
                                                nowy_tż,
                                                100,
                                                0,
                                                0,
                                                nowy_fenotyp_a,
                                                nowy_fenotyp_b,
                                                nowa_lokalizacja,
                                                0]]),
                                axis=0)
                Gra_zdrajcy = np.append(Gra_zdrajcy,
                                        np.zeros((1, Gra_zdrajcy.shape[1])),
                                        axis = 0)
                Gra_zdrajcy = np.append(Gra_zdrajcy,
                                        np.zeros((Gra_zdrajcy.shape[0], 1)),
                                        axis = 1)
                
                Gra_rodzice = np.append(Gra_rodzice,
                                        np.zeros((1, Gra_rodzice.shape[1])),
                                        axis = 0)
                Gra_rodzice = np.append(Gra_rodzice,
                                        np.zeros((Gra_rodzice.shape[0], 1)),
                                        axis = 1)
                
                Gra_potomkowie = np.append(Gra_potomkowie,
                                        np.zeros((1, Gra_potomkowie.shape[1])),
                                        axis = 0)
                Gra_potomkowie = np.append(Gra_potomkowie,
                                        np.zeros((Gra_potomkowie.shape[0], 1)),
                                        axis = 1)
                Gra_rodzice[nowy_index, int(indeksy[q2])] = 1
                Gra_rodzice[nowy_index, int(indeksy[q2+1])] = 1
                
                Gra_potomkowie[int(indeksy[q2]), nowy_index] = 1
                Gra_potomkowie[int(indeksy[q2+1]), nowy_index] = 1
        except:
            continue
    #Zapisanie wyników cząskowych
    Wyniki_część = pd.DataFrame(Gra)
    Wyniki_część["Runda"] = _+1
    Wyniki_całość = pd.concat([Wyniki_całość, Wyniki_część], ignore_index=True)
Wyniki_całość.columns = ["Indeks", "Strategia bazowa", "Pamięć", "Tryb życia", "Punkty",
                         "Wiek", "Status", "Fenotyp A", "Fenotyp B", "Lokalizacja",
                         "Powód śmierci", "Runda"]
execution_time = time.time() - start_time
print(f"Czas wykonania pętli: {round(execution_time, 2)} sekundy")

Runda 1
Runda 2
Runda 3
Runda 4
Runda 5
Runda 6
Runda 7
Runda 8
Runda 9
Runda 10
Czas wykonania pętli: 0.5 sekundy


In [24]:
Wyniki_całość

,Indeks,Strategia bazowa,Pamięć,Tryb życia,Punkty,Wiek,Status,Fenotyp A,Fenotyp B,Lokalizacja,Powód śmierci,Runda
0,0.0,0.0,1.0,0.0,98.04,1.0,0.0,0.0000,100.0000,3.0,0.0,0
1,1.0,0.0,0.0,0.0,158.08,1.0,0.0,100.0000,0.0000,2.0,0.0,0
2,2.0,2.0,0.0,1.0,78.14,1.0,0.0,100.0000,0.0000,0.0,0.0,0
3,3.0,5.0,1.0,0.0,80.11,1.0,0.0,100.0000,0.0000,1.0,0.0,0
4,4.0,3.0,0.0,0.0,120.02,1.0,0.0,100.0000,0.0000,2.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
590,106.0,4.0,1.0,0.0,100.00,0.0,0.0,18.7500,81.2500,2.0,0.0,10
591,107.0,4.0,1.0,0.0,100.00,0.0,0.0,17.1875,82.8125,2.0,0.0,10
592,108.0,4.0,1.0,1.0,100.00,0.0,0.0,21.8750,78.1250,2.0,0.0,10
593,109.0,4.0,1.0,0.0,100.00,0.0,0.0,0.0000,100.0000,2.0,0.0,10


In [25]:
Wyniki_całość["Indeks"] = Wyniki_całość["Indeks"].astype(int)
Wyniki_całość["Strategia bazowa"] = Wyniki_całość["Strategia bazowa"].replace(0, "E")
Wyniki_całość["Strategia bazowa"] = Wyniki_całość["Strategia bazowa"].replace(1, "K")
Wyniki_całość["Strategia bazowa"] = Wyniki_całość["Strategia bazowa"].replace(2, "P")
Wyniki_całość["Strategia bazowa"] = Wyniki_całość["Strategia bazowa"].replace(3, "R")
Wyniki_całość["Strategia bazowa"] = Wyniki_całość["Strategia bazowa"].replace(4, "F")
Wyniki_całość["Strategia bazowa"] = Wyniki_całość["Strategia bazowa"].replace(5, "A")
Wyniki_całość["Pamięć"] = Wyniki_całość["Pamięć"].replace(0, "B")
Wyniki_całość["Pamięć"] = Wyniki_całość["Pamięć"].replace(1, "D")
Wyniki_całość["Tryb życia"] = Wyniki_całość["Tryb życia"].replace(0, "K")
Wyniki_całość["Tryb życia"] = Wyniki_całość["Tryb życia"].replace(1, "Z")
Wyniki_całość["Wiek"] = Wyniki_całość["Wiek"].astype(int)
Wyniki_całość["Status"] = Wyniki_całość["Status"].replace(0, "Ż")
Wyniki_całość["Status"] = Wyniki_całość["Status"].replace(1, "Ś")
Wyniki_całość["Lokalizacja"] = Wyniki_całość["Lokalizacja"].astype(int)
Wyniki_całość["Powód śmierci"] = Wyniki_całość["Powód śmierci"].replace(0, "")
Wyniki_całość["Powód śmierci"] = Wyniki_całość["Powód śmierci"].replace(1, "N")
Wyniki_całość["Powód śmierci"] = Wyniki_całość["Powód śmierci"].replace(2, "W")
Wyniki_całość["Strategia synkretyczna"] = Wyniki_całość["Strategia bazowa"] + Wyniki_całość["Pamięć"] + Wyniki_całość["Tryb życia"]
Wyniki_całość

,Indeks,Strategia bazowa,Pamięć,Tryb życia,Punkty,Wiek,Status,Fenotyp A,Fenotyp B,Lokalizacja,Powód śmierci,Runda,Strategia synkretyczna
0,0,E,D,K,98.04,1,Ż,0.0000,100.0000,3,,0,EDK
1,1,E,B,K,158.08,1,Ż,100.0000,0.0000,2,,0,EBK
2,2,P,B,Z,78.14,1,Ż,100.0000,0.0000,0,,0,PBZ
3,3,A,D,K,80.11,1,Ż,100.0000,0.0000,1,,0,ADK
4,4,R,B,K,120.02,1,Ż,100.0000,0.0000,2,,0,RBK
...,...,...,...,...,...,...,...,...,...,...,...,...,...
590,106,F,D,K,100.00,0,Ż,18.7500,81.2500,2,,10,FDK
591,107,F,D,K,100.00,0,Ż,17.1875,82.8125,2,,10,FDK
592,108,F,D,Z,100.00,0,Ż,21.8750,78.1250,2,,10,FDZ
593,109,F,D,K,100.00,0,Ż,0.0000,100.0000,2,,10,FDK


In [26]:
Wyniki_k = pd.DataFrame(Gra[:, 1:],
                        index = Gra[:,0].astype(int),
                        columns = ["Strategia bazowa",
                                       "Pamięć",
                                       "Tryb życia",
                                       "Punkty",
                                       "Wiek",
                                       "Status",
                                       "Fenotyp A",
                                       "Fenotyp B",
                                       "Lokalizacja",
                                       "Powód śmierci"])

In [27]:
Wyniki_k["Strategia bazowa"] = Wyniki_k["Strategia bazowa"].replace(0, "E")
Wyniki_k["Strategia bazowa"] = Wyniki_k["Strategia bazowa"].replace(1, "K")
Wyniki_k["Strategia bazowa"] = Wyniki_k["Strategia bazowa"].replace(2, "P")
Wyniki_k["Strategia bazowa"] = Wyniki_k["Strategia bazowa"].replace(3, "R")
Wyniki_k["Strategia bazowa"] = Wyniki_k["Strategia bazowa"].replace(4, "F")
Wyniki_k["Strategia bazowa"] = Wyniki_k["Strategia bazowa"].replace(5, "A")
Wyniki_k["Pamięć"] = Wyniki_k["Pamięć"].replace(0, "B")
Wyniki_k["Pamięć"] = Wyniki_k["Pamięć"].replace(1, "D")
Wyniki_k["Tryb życia"] = Wyniki_k["Tryb życia"].replace(0, "K")
Wyniki_k["Tryb życia"] = Wyniki_k["Tryb życia"].replace(1, "Z")
Wyniki_k["Wiek"] = Wyniki_k["Wiek"].astype(int)
Wyniki_k["Status"] = Wyniki_k["Status"].replace(0, "Ż")
Wyniki_k["Status"] = Wyniki_k["Status"].replace(1, "Ś")
Wyniki_k["Lokalizacja"] = Wyniki_k["Lokalizacja"].astype(int)
Wyniki_k["Powód śmierci"] = Wyniki_k["Powód śmierci"].replace(0, "")
Wyniki_k["Powód śmierci"] = Wyniki_k["Powód śmierci"].replace(1, "N")
Wyniki_k["Powód śmierci"] = Wyniki_k["Powód śmierci"].replace(2, "W")
Wyniki_k["Strategia synkretyczna"] = Wyniki_k["Strategia bazowa"] + Wyniki_k["Pamięć"] + Wyniki_k["Tryb życia"]
Wyniki_k

,Strategia bazowa,Pamięć,Tryb życia,Punkty,Wiek,Status,Fenotyp A,Fenotyp B,Lokalizacja,Powód śmierci,Strategia synkretyczna
0,E,D,K,135.40,10,Ż,0.0000,100.0000,2,,EDK
1,E,B,K,66.44,10,Ż,100.0000,0.0000,2,,EBK
2,P,B,Z,-9.75,2,Ś,100.0000,0.0000,0,W,PBZ
3,A,D,K,-0.58,5,Ś,100.0000,0.0000,0,W,ADK
4,R,B,K,-2.65,5,Ś,100.0000,0.0000,2,W,RBK
...,...,...,...,...,...,...,...,...,...,...,...
106,F,D,K,100.00,0,Ż,18.7500,81.2500,2,,FDK
107,F,D,K,100.00,0,Ż,17.1875,82.8125,2,,FDK
108,F,D,Z,100.00,0,Ż,21.8750,78.1250,2,,FDZ
109,F,D,K,100.00,0,Ż,0.0000,100.0000,2,,FDK


In [28]:
test1 = pd.DataFrame(Gra_potomkowie)
Wyniki_k["Potomkowie"] = test1.apply(lambda row: np.where(row == 1)[0], axis=1)
Wyniki_k["Potomkowie"] = Wyniki_k["Potomkowie"].apply(lambda x: str(x).replace("[", "").replace("]", ""))
test2 = pd.DataFrame(Gra_rodzice)
Wyniki_k["Rodzice"] = test2.apply(lambda row: np.where(row == 1)[0], axis=1)
Wyniki_k["Rodzice"] = Wyniki_k["Rodzice"].apply(lambda x: str(x).replace("[", "").replace("]", ""))
test3 = pd.DataFrame(Gra_zdrajcy)
Wyniki_k["Zdrajcy"] = test3.apply(lambda row: np.where(row == 1)[0], axis=1)
Wyniki_k["Zdrajcy"] = Wyniki_k["Zdrajcy"].apply(lambda x: str(x).replace("[", "").replace("]", ""))
Wyniki_k

,Strategia bazowa,Pamięć,Tryb życia,Punkty,Wiek,Status,Fenotyp A,Fenotyp B,Lokalizacja,Powód śmierci,Strategia synkretyczna,Potomkowie,Rodzice,Zdrajcy
0,E,D,K,135.40,10,Ż,0.0000,100.0000,2,,EDK,25 44 50 61 65 86 106,,1 11 13 18 23 24 25 27 28 30 33 36 37 38 40 4...
1,E,B,K,66.44,10,Ż,100.0000,0.0000,2,,EBK,28 45 77,,0 5 7 8 10 11 14 15 18 21 22 23 24 28 29 3...
2,P,B,Z,-9.75,2,Ś,100.0000,0.0000,0,W,PBZ,20,,1 4 5 7 11 14 16 18 20 22
3,A,D,K,-0.58,5,Ś,100.0000,0.0000,0,W,ADK,,,0 1 5 8 10 11 18 21 22 23 24 25 26 28 29 35
4,R,B,K,-2.65,5,Ś,100.0000,0.0000,2,W,RBK,22,,0 7 8 10 11 15 18 21 23 25 28 29 33 34 36 3...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,F,D,K,100.00,0,Ż,18.7500,81.2500,2,,FDK,,0 47,
107,F,D,K,100.00,0,Ż,17.1875,82.8125,2,,FDK,,43 79,
108,F,D,Z,100.00,0,Ż,21.8750,78.1250,2,,FDZ,,58 61,
109,F,D,K,100.00,0,Ż,0.0000,100.0000,2,,FDK,,44 49,


In [29]:
Wyniki_badania = pd.ExcelWriter("Wyniki_badania.xlsx")
Wyniki_całość.to_excel(Wyniki_badania, sheet_name="Wyniki szczegółowe")
Wyniki_k.to_excel(Wyniki_badania, sheet_name="Wyniki końcowe")
Wyniki_badania.save()

In [30]:
Wyniki_k.dtypes

Strategia bazowa           object
Pamięć                     object
Tryb życia                 object
Punkty                    float64
Wiek                        int32
Status                     object
Fenotyp A                 float64
Fenotyp B                 float64
Lokalizacja                 int32
Powód śmierci              object
Strategia synkretyczna     object
Potomkowie                 object
Rodzice                    object
Zdrajcy                    object
dtype: object

In [31]:
Gra.shape

(111, 11)